# Finding the Best Hyperparameters for BERTopic

In this notebook we will try to find the best hyperparameters for our BERTopic model, by trying different configurations of UMAP and HDBSCAN models. Then we will evaluate each model based on both standard evaluation metrics and manual inspection of the topics created. 

## 

## Imports & Setup

In [12]:
import pandas as pd 
import glob
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer

from utils.data_loader import DataLoader
from utils.bertopic_evaluator import BERTopicModelEvaluator
from config.model import NUM_TOPICS, TOP_K, EMBEDDING_MODEL, vectorizer_params, c_tfidf_params, metrics
from config.optimization import all_config_combinations, algos_dict

In [ ]:
loader = DataLoader('../../data/data_speeches.csv', '../../data/data_statements.csv', split_data=False)
loader.process()

train_docs, train_sentences = loader.get_train_data()

For this step, we only need the training and validation data.

## Constant Model Initialization

We will use the same vectorizer, c-TF-IDF model and sentence transformer model for all our experiments. 

We're interesting in optimizing on the Dimensionality Reduction and Clustering models, so we will keep the rest of the pipeline constant.

In [ ]:
vectorizer_model = CountVectorizer(**vectorizer_params)
ctfidf_model = ClassTfidfTransformer(**c_tfidf_params)
st_model = SentenceTransformer(EMBEDDING_MODEL)

Evaluating different UMAP and HDBSCAN configurations

## Hyperparameter Tuning

Let's see how many different configurations we can try:

In [ ]:
search_space = all_config_combinations()
len(search_space)

Start from checkpointed model - paste last model that was being trained before the crash as a string. Example in the cell below.

In [ ]:
# start_from = "model_UMAP_{'n_components': 30, 'n_neighbors': 5, 'min_dist': 0.1}_HDBSCAN_{'min_cluster_size': 20, 'metric': 'euclidean', 'prediction_data': False}"

Do not touch, **uncomment only if you want to start from a checkpointed model**.

In [ ]:
# for i, config in enumerate(search_space):
#     model_name = f"model_{config['dim_reduction_model']}_{config['dim_reduction_params']}_{config['clustering_model']}_{config['clustering_params']}"
#     if model_name == start_from:
#         start_from = i
#         break
    
# print(f"Starting from {start_from}")
# search_space = search_space[start_from:]

### Document Level

In [ ]:
granularity = 'docs'

In [ ]:
models = {}

print("Loading models...")

for config in search_space:
    dim_reduction_model = algos_dict[config['dim_reduction_model']](**config['dim_reduction_params'])
    clustering_model = algos_dict[config['clustering_model']](**config['clustering_params'])
        
    model_name = f"model_{config['dim_reduction_model']}_{config['dim_reduction_params']}_{config['clustering_model']}_{config['clustering_params']}"


    model = BERTopic(
        umap_model=dim_reduction_model,
        hdbscan_model=clustering_model,
        embedding_model=EMBEDDING_MODEL,
        vectorizer_model=vectorizer_model,
        ctfidf_model=ctfidf_model,
        nr_topics=NUM_TOPICS,
        top_n_words=TOP_K,
    )

    models[model_name] = model

In [ ]:
evaluator = BERTopicModelEvaluator(
                                   models=models, 
                                   metrics=metrics, 
                                   dataset=train_docs,
                                   topics=NUM_TOPICS,
                                   top_k=TOP_K,
                                   granularity=granularity
                                   )

In [ ]:
print("Evaluating models...")
evaluator.evaluate()

### Sentence Level

In [ ]:
granularity = 'sent'

In [ ]:
models = {}

print("Loading models...")

for config in search_space:
    dim_reduction_model = algos_dict[config['dim_reduction_model']](**config['dim_reduction_params'])
    clustering_model = algos_dict[config['clustering_model']](**config['clustering_params'])
        
    model_name = f"model_{config['dim_reduction_model']}_{config['dim_reduction_params']}_{config['clustering_model']}_{config['clustering_params']}"


    model = BERTopic(
        umap_model=dim_reduction_model,
        hdbscan_model=clustering_model,
        embedding_model=EMBEDDING_MODEL,
        vectorizer_model=vectorizer_model,
        ctfidf_model=ctfidf_model,
        nr_topics=NUM_TOPICS,
        top_n_words=TOP_K,
    )

    models[model_name] = model

In [ ]:
evaluator = BERTopicModelEvaluator(
                                   models=models, 
                                   metrics=metrics, 
                                   dataset=train_docs,
                                   topics=NUM_TOPICS,
                                   top_k=TOP_K,
                                   granularity=granularity
                                   )

In [ ]:
print("Evaluating models...")
evaluator.evaluate()

## Best Model Selection

### Document Level

In [34]:
# open all csvs and merge them into one
doc_res_df = pd.DataFrame()

doc_res_path = "data/optimization/docs_gran/individual/*.csv"

for csv in glob.glob(doc_res_path):
    df = pd.read_csv(csv)
    doc_res_df = pd.concat([doc_res_df, df])

doc_res_df = doc_res_df.drop("Unnamed: 0", axis=1).drop("model.1", axis=1)
doc_res_df.dropna(inplace=True)
doc_res_df = doc_res_df.sort_values(by=['coherence_c_v', "diversity_topic"], ascending=False)
doc_res_df.to_csv("data/optimization/docs_gran/doc_results.csv", index=False)

doc_res_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 114 entries, 0 to 0
Data columns (total 37 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   model             114 non-null    object 
 1   coherence_c_npmi  114 non-null    float64
 2   coherence_c_v     114 non-null    float64
 3   coherence_u_mass  114 non-null    float64
 4   coherence_c_uci   114 non-null    float64
 5   diversity_topic   114 non-null    float64
 6   similarity_rbo    114 non-null    float64
 7   similarity_pjs    114 non-null    float64
 8   topic_0           114 non-null    object 
 9   topic_1           114 non-null    object 
 10  topic_2           114 non-null    object 
 11  topic_3           114 non-null    object 
 12  topic_4           114 non-null    object 
 13  topic_5           114 non-null    object 
 14  topic_6           114 non-null    object 
 15  topic_7           114 non-null    object 
 16  topic_8           114 non-null    object 
 17  topi

In [39]:
doc_top10_df = doc_res_df.head(10).copy()
doc_top10_df

,model,coherence_c_npmi,coherence_c_v,coherence_u_mass,coherence_c_uci,diversity_topic,similarity_rbo,similarity_pjs,topic_0,topic_1,...,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27,topic_28
0,"model_UMAP_{'n_components': 10, 'n_neighbors':...",0.093442,0.668072,-0.471180,-1.079063,0.926108,0.005254,0.003511,"['μητσοτάκη', 'κύριε μητσοτάκη', 'εκλογές', 'ε...","['υγείας', 'νοσοκομείο', 'σύστημα υγείας', 'νο...",...,"['σχολείο', 'μαθητές', 'δασκάλους', 'καθηγητές...","['αερίου', 'φυσικού αερίου', 'φυσικού', 'βουλγ...","['τουρισμού', 'τουρισμό', 'τουρισμός', 'τουρισ...","['καναδά', 'brad', 'brad smith', 'justin', 'sm...","['εαβ', 'αεροπορίας', 'πολεμικής αεροπορίας', ...","['λιμενικού', 'λιμενικού σώματος', 'ναυτιλία',...","['γαλλίας', 'γαλλία', 'de', 'nous', 'et', 'la'...","['νατο', 'alliance', 'nato', 'thank', 'securit...","['πανεπιστήμιο', 'εκπαίδευση', 'πανεπιστήμια',...","['ρωσικές', 'οασε', 'ρωσικές σχέσεις', 'φιντέλ..."
0,"model_UMAP_{'n_components': 15, 'n_neighbors':...",0.088311,0.663455,-0.582842,-1.261581,0.916256,0.006136,0.003796,"['ευρώ', 'εκλογές', 'τσίπρα', 'οικονομία', 'μη...","['υγείας', 'νοσοκομείο', 'σύστημα υγείας', 'νο...",...,"['λιμενικού', 'λιμενικού σώματος', 'ναυτιλία',...","['αερίου', 'βουλγαρίας', 'βουλγαρία', 'φυσικού...","['εαβ', 'αεροπορίας', 'ενόπλων δυνάμεων', 'ενό...","['υγείας', 'ψυχικής υγείας', 'ψυχικής', 'εμβόλ...","['μπορούμε μιλάμε', 'οικονομικό χρηματοπιστωτι...","['πανεπιστήμιο', 'παιδί', 'δημογραφικό', 'σχολ...","['γαλλίας', 'γαλλία', 'nous', 'de', 'et', 'la'...","['alliance', 'νατο', 'thank', 'security', 'nat...","['γυναικών', 'γυναίκες', 'βίας', 'γυναίκας', '...","['δεθ', 'θεσσαλονίκης', 'θεσσαλονίκη', 'έκθεση..."
0,"model_UMAP_{'n_components': 30, 'n_neighbors':...",0.081087,0.655294,-0.485989,-1.280914,0.950739,0.003626,0.002147,"['οικονομία', 'τσίπρα', 'εκλογές', 'ευρώπη', '...","['ευρωπαϊκή', 'ευρώπη', 'ευρωπαϊκής', 'συνεργα...",...,"['εαβ', 'αεροπορίας', 'πολεμικής αεροπορίας', ...","['μετρό', 'έργα', 'γραμμής', 'θεσσαλονίκης', '...","['τεκα', 'εργασίας', 'ασφάλισης', 'ασφάλιση', ...","['γυναίκες', 'γυναικών', 'βίας', 'γυναίκας', '...","['γαλλίας', 'nous', 'de', 'et', 'γαλλία', 'la'...","['αναθεώρησης συντάγματος', 'ενωμένοι όρθιοι',...","['λεωφορεία', 'μαζικής μεταφοράς', 'leasing', ...","['εκκλησίας', 'παναγίου', 'παναγίου τάφου', 'θ...","['μάχη', 'νυχτωμένοι πιστεύουν', 'συντηρητικοί...","['δεθ', 'θεσσαλονίκης', 'θεσσαλονίκη', 'έκθεση..."
0,"model_UMAP_{'n_components': 20, 'n_neighbors':...",0.081927,0.648560,-0.573473,-1.358022,0.935961,0.004917,0.002747,"['ευρώ', 'τσίπρα', 'εκλογές', 'μητσοτάκη', 'κύ...","['υγείας', 'νοσοκομείο', 'σύστημα υγείας', 'νο...",...,"['τουρισμού', 'τουρισμό', 'τουρισμός', 'τουρισ...","['οικοσύστημα', 'τεχνολογίας', 'εταιρείες', 'c...","['γαλλίας', 'γαλλία', 'de', 'nous', 'et', 'la'...","['λιμενικού', 'λιμενικού σώματος', 'ναυτιλία',...","['καναδά', 'brad', 'brad smith', 'justin', 'sm...","['εαβ', 'αεροπορίας', 'viper', 'ενόπλων δυνάμε...","['alliance', 'νατο', 'thank', 'security', 'nat...","['μουσείο', 'αρχαιολογικό', 'πολιτισμού', 'αιγ...","['υγείας', 'εμβόλια', 'john micklethwait', 'mi...","['πανεπιστήμια', 'πανεπιστήμιο', 'πανεπιστημίω..."
0,"model_UMAP_{'n_components': 5, 'n_neighbors': ...",0.069568,0.647382,-0.499941,-1.605552,0.916256,0.007416,0.004143,"['τσίπρα', 'μητσοτάκη', 'κύριε μητσοτάκη', 'εκ...","['υγείας', 'νοσοκομείο', 'σύστημα υγείας', 'νο...",...,"['τουρισμού', 'τουρισμό', 'τουρισμός', 'σετε',...","['υγείας', 'ψυχικής', 'ψυχικής υγείας', 'εμβόλ...","['λιμενικού', 'λιμενικού σώματος', 'ναυτιλία',...","['έργα', 'δρόμος', 'έργου', 'πάτρα', 'τμήμα', ...","['αερίου', 'φυσικού αερίου', 'ενέργειας', 'φυσ...","['αερίου', 'φυσικού αερίου', 'βουλγαρίας', 'φυ...","['γυναικών', 'γυναίκες', 'βίας', 'λοατκι', 'εν...","['γαλλίας', 'γαλλία', 'nous', 'de', 'et', 'la'...","['alliance', 'νατο', 'thank', 'security', 'nat...","['μουσείο', 'αιγών', 'αρχαιολογικό', 'πολιτισμ..."
0,"model_UMAP_{'n_components': 15, 'n_

In [40]:
doc_top10_df.to_csv("data/optimization/docs_gran/doc_top10.csv", index=False)

Model 6 looks like the overall best model, with a good balance between coherence and diversity, as well as well-defined topics.

In [43]:
doc_best_model = doc_top10_df.iloc[5]["model"]
doc_best_model

"model_UMAP_{'n_components': 15, 'n_neighbors': 15, 'min_dist': 0.2}_HDBSCAN_{'min_cluster_size': 7, 'metric': 'euclidean', 'prediction_data': False}"

: 